In [1]:
FALL = "2017"
SPRING = str(int(FALL) + 1)

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("Data/games_cleaned_" + FALL + "_" + SPRING + ".tsv", sep="\t", index_col=None)
df.date = df.date.astype("str")
df["spread_bet"] = "NO BET"
df["ml_bet"] = "NO BET"
df.dtypes

date               object
day                 int64
away               object
home               object
opening_spread    float64
away_ML             int64
home_ML             int64
away_pts            int64
home_pts            int64
result              int64
result_diff       float64
spread_bet         object
ml_bet             object
dtype: object

In [4]:
teams_df = pd.DataFrame(columns=["wins", "losses", "total", "pct", "ptDiff_total", "ptDiff"], index=df.away.unique())
teams_df = teams_df.sort_index()
teams_df[["wins", "losses", "ptDiff_total"]] = teams_df[["wins", "losses", "ptDiff_total"]].fillna(0)
teams_df

wins  losses total  pct  ptDiff_total ptDiff
Atlanta           0       0   NaN  NaN             0    NaN
Boston            0       0   NaN  NaN             0    NaN
Brooklyn          0       0   NaN  NaN             0    NaN
Charlotte         0       0   NaN  NaN             0    NaN
Chicago           0       0   NaN  NaN             0    NaN
Cleveland         0       0   NaN  NaN             0    NaN
Dallas            0       0   NaN  NaN             0    NaN
Denver            0       0   NaN  NaN             0    NaN
Detroit           0       0   NaN  NaN             0    NaN
Golden State      0       0   NaN  NaN             0    NaN
Houston           0       0   NaN  NaN             0    NaN
Indiana           0       0   NaN  NaN             0    NaN
L.A. Clippers     0       0   NaN  NaN             0    NaN
L.A. Lakers       0       0   NaN  NaN             0    NaN
Memphis           0       0   NaN  NaN             0    NaN
Miami             0       0   NaN  NaN             0    NaN
Milwaukee         0       0   NaN  NaN             0    NaN
Minnesota         0       0   NaN  NaN             0    NaN
New Orleans       0       0   NaN  NaN             0    NaN
New York          0       0   NaN  NaN             0    NaN
Oklahoma City     0       0   NaN  NaN             0    NaN
Orlando           0       0   NaN  NaN             0    NaN
Philadelphia      0       0   NaN  NaN             0    NaN
Phoenix           0       0   NaN  NaN             0    NaN
Portland          0       0   NaN  NaN             0    NaN
Sacramento        0       0   NaN  NaN             0    NaN
San Antonio       0       0   NaN  NaN             0    NaN
Toronto           0       0   NaN  NaN             0    NaN
Utah              0       0   NaN  NaN             0    NaN
Washington        0       0   NaN  NaN             0    NaN

In [5]:
# takes a DataFrame of all games from 1 day
def updateStats(today_df):
    for index, row in today_df.iterrows():
        
        if row["result"] < 0:  # home wins
            teams_df.at[row["home"], "wins"] += 1
            teams_df.at[row["away"], "losses"] += 1
            
        elif row["result"] > 0:  # away wins
            teams_df.at[row["away"], "wins"] += 1
            teams_df.at[row["home"], "losses"] += 1

        else:
            print("tie???")
            assert False
        
        teams_df.at[row["home"], "ptDiff_total"] += (-1 * row["result"])
        teams_df.at[row["away"], "ptDiff_total"] += row["result"]
    
    teams_df["total"] = teams_df.wins + teams_df.losses
    teams_df["pct"] = teams_df.wins / (teams_df.wins + teams_df.losses)
    
    teams_df["ptDiff"] = teams_df.ptDiff_total / teams_df.total
    
    return teams_df

In [6]:
# SPREAD BET FUNCTIONS
# must take a row of df as argument
# can reference teams_df since it's global

def fadeLAL(r):
    if r["away"] == "L.A. Lakers":
        return "home"
    elif r["home"] == "L.A. Lakers":
        return "away"
    else:
        return "NO BET"

def fadeHomeFavs(r):
    if r["opening_spread"] <= -3.0:
        return "away"
    else:
        return "NO BET"

In [7]:
# ML BET FUNCTIONS
# work same as spread bet functions

def pickFavs(r):
    if r["opening_spread"] < 0:
        return "home"
    elif r["opening_spread"] > 0:
        return "away"

def pickRecord(r):
    home = r["home"]
    away = r["away"]
    
    if teams_df.at[home, "pct"] > teams_df.at[away, "pct"]:
        return "home"
    elif teams_df.at[away, "pct"] > teams_df.at[home, "pct"]:
        return "away"
    else:
        return "NO BET"

def pickPtDiff(r):
    home = r["home"]
    away = r["away"]
    cushion = 0.1
    
    if teams_df.at[home, "ptDiff"] + cushion > teams_df.at[away, "ptDiff"]:
        return "home"
    elif teams_df.at[away, "ptDiff"] + cushion > teams_df.at[home, "ptDiff"]:
        return "away"
    else:
        return "NO BET"

In [8]:
# simulate season and place bets with chosen bet functions and parameters

startDate = "20171101"
endDate = "20180411"
spread_betFunction = fadeHomeFavs
ml_betFunction = pickFavs

for index, row in df.iterrows():
    # if new day, update to-date stats for yesterday's games
    if row["day"] != 0 and row["day"] != df.at[index-1, "day"]:
        updateStats(df[df.day==df.at[index-1, "day"]])
    
    # if end day, break
    if endDate:
        if row["date"] == endDate:
            updateStats(df[df.day==row["day"]])  # update stats for the final day of games, both for debugging reasons and for the sake of my OCD
            break
    
    # retrain algorithm
    
    if row["date"] >= startDate:
        df.at[index, "spread_bet"] = spread_betFunction(row)
        df.at[index, "ml_bet"] = ml_betFunction(row)

df

date  day           away           home  opening_spread  away_ML  \
0     20171017    0         Boston      Cleveland            -3.0      155   
1     20171017    0        Houston   Golden State            -9.0      385   
2     20171018    1       Brooklyn        Indiana            -3.5      145   
3     20171018    1          Miami        Orlando             3.0     -150   
4     20171018    1      Milwaukee         Boston            -5.5      205   
5     20171018    1        Atlanta         Dallas            -6.5      220   
6     20171018    1      Minnesota    San Antonio            -1.0      105   
7     20171018    1   Philadelphia     Washington            -7.0      255   
8     20171018    1      Charlotte        Detroit            -3.0      145   
9     20171018    1    New Orleans        Memphis            -3.0      135   
10    20171018    1         Denver           Utah            -2.0      110   
11    20171018    1        Houston     Sacramento             8.5     -360   
12    20171018    1       Portland        Phoenix             1.0     -123   
13    20171019    2        Chicago        Toronto           -11.0      750   
14    20171019    2       New York  Oklahoma City           -11.0      575   
15    20171019    2  L.A. Clippers    L.A. Lakers             4.5     -195   
16    20171020    3         Boston   Philadelphia             1.0      105   
17    20171020    3        Detroit     Washington            -6.5      200   
18    20171020    3       Portland        Indiana             3.5     -165   
19    20171020    3           Utah      Minnesota            -5.5      165   
20    20171020    3   Golden State    New Orleans             6.5     -320   
21    20171020    3        Atlanta      Charlotte            -6.0      200   
22    20171020    3      Cleveland      Milwaukee             1.5     -135   
23    20171020    3        Orlando       Brooklyn            -1.5      105   
24    20171020    3     Sacramento         Dallas            -5.5      180   
25    20171020    3    L.A. Lakers        Phoenix            -3.0      130   
26    20171021    4   Philadelphia        Toronto            -7.5      415   
27    20171021    4         Dallas        Houston           -10.5      700   
28    20171021    4        Indiana          Miami            -8.5      425   
29    20171021    4   Golden State        Memphis             7.5     -440   
...        ...  ...            ...            ...             ...      ...   
1200  20180408  173        Detroit        Memphis             5.5     -220   
1201  20180408  173           Utah    L.A. Lakers             6.0     -325   
1202  20180408  173   Golden State        Phoenix            13.5    -1150   
1203  20180409  174        Toronto        Detroit             3.5     -175   
1204  20180409  174        Chicago       Brooklyn            -8.5      290   
1205  20180409  174        Memphis      Minnesota           -15.0     1000   
1206  20180409  174     Sacramento    San Antonio           -13.0      850   
1207  20180409  174      Cleveland       New York            12.0     -800   
1208  20180409  174  Oklahoma City          Miami             2.0     -140   
1209  20180409  174        Orlando      Milwaukee           -10.0      550   
1210  20180409  174       Portland         Denver            -3.5      155   
1211  20180409  174    New Orleans  L.A. Clippers             5.5     -255   
1212  20180410  175      Charlotte        Indiana            -6.5      240   
1213  20180410  175         Boston     Washington            -6.5      245   
1214  20180410  175   Golden State           Utah            -7.0      245   
1215  20180410  175   Philadelphia        Atlanta             9.5     -500   
1216  20180410  175        Phoenix         Dallas            -4.0      150   
1217  20180410  175        Houston    L.A. Lakers             9.5     -660   
1218  20180411  176        Toronto          Miami            -7.0      170   
1219  20180411  176      Milwauk

In [9]:
teams_df

wins  losses  total       pct  ptDiff_total    ptDiff
Atlanta          24      58     82  0.292683          -447 -5.451220
Boston           55      27     82  0.670732           294  3.585366
Brooklyn         28      54     82  0.341463          -307 -3.743902
Charlotte        36      46     82  0.439024            21  0.256098
Chicago          27      55     82  0.329268          -577 -7.036585
Cleveland        50      32     82  0.609756            77  0.939024
Dallas           24      58     82  0.292683          -249 -3.036585
Denver           46      36     82  0.560976           121  1.475610
Detroit          39      43     82  0.475610           -12 -0.146341
Golden State     58      24     82  0.707317           490  5.975610
Houston          65      17     82  0.792683           695  8.475610
Indiana          48      34     82  0.585366           113  1.378049
L.A. Clippers    42      40     82  0.512195             3  0.036585
L.A. Lakers      35      47     82  0.426829          -127 -1.548780
Memphis          22      60     82  0.268293          -509 -6.207317
Miami            44      38     82  0.536585            39  0.475610
Milwaukee        44      38     82  0.536585           -25 -0.304878
Minnesota        47      35     82  0.573171           183  2.231707
New Orleans      48      34     82  0.585366           107  1.304878
New York         29      53     82  0.353659          -292 -3.560976
Oklahoma City    48      34     82  0.585366           280  3.414634
Orlando          25      57     82  0.304878          -395 -4.817073
Philadelphia     52      30     82  0.634146           369  4.500000
Phoenix          21      61     82  0.256098          -768 -9.365854
Portland         49      33     82  0.597561           213  2.597561
Sacramento       27      55     82  0.329268          -573 -6.987805
San Antonio      47      35     82  0.573171           237  2.890244
Toronto          59      23     82  0.719512           638  7.780488
Utah             48      34     82  0.585366           353  4.304878
Washington       43      39     82  0.524390            48  0.585366

In [10]:
# grade results of the bets

df["spread_bet_result"] = "N/A"

df.loc[(df.spread_bet == "home") & (df.result_diff < 0), "spread_bet_result"] = "W"
df.loc[(df.spread_bet == "home") & (df.result_diff > 0), "spread_bet_result"] = "L"
df.loc[(df.spread_bet == "home") & (df.result_diff == 0), "spread_bet_result"] = "P"
df.loc[(df.spread_bet == "away") & (df.result_diff < 0), "spread_bet_result"] = "L"
df.loc[(df.spread_bet == "away") & (df.result_diff > 0), "spread_bet_result"] = "W"
df.loc[(df.spread_bet == "away") & (df.result_diff == 0), "spread_bet_result"] = "P"

df

date  day           away           home  opening_spread  away_ML  \
0     20171017    0         Boston      Cleveland            -3.0      155   
1     20171017    0        Houston   Golden State            -9.0      385   
2     20171018    1       Brooklyn        Indiana            -3.5      145   
3     20171018    1          Miami        Orlando             3.0     -150   
4     20171018    1      Milwaukee         Boston            -5.5      205   
5     20171018    1        Atlanta         Dallas            -6.5      220   
6     20171018    1      Minnesota    San Antonio            -1.0      105   
7     20171018    1   Philadelphia     Washington            -7.0      255   
8     20171018    1      Charlotte        Detroit            -3.0      145   
9     20171018    1    New Orleans        Memphis            -3.0      135   
10    20171018    1         Denver           Utah            -2.0      110   
11    20171018    1        Houston     Sacramento             8.5     -360   
12    20171018    1       Portland        Phoenix             1.0     -123   
13    20171019    2        Chicago        Toronto           -11.0      750   
14    20171019    2       New York  Oklahoma City           -11.0      575   
15    20171019    2  L.A. Clippers    L.A. Lakers             4.5     -195   
16    20171020    3         Boston   Philadelphia             1.0      105   
17    20171020    3        Detroit     Washington            -6.5      200   
18    20171020    3       Portland        Indiana             3.5     -165   
19    20171020    3           Utah      Minnesota            -5.5      165   
20    20171020    3   Golden State    New Orleans             6.5     -320   
21    20171020    3        Atlanta      Charlotte            -6.0      200   
22    20171020    3      Cleveland      Milwaukee             1.5     -135   
23    20171020    3        Orlando       Brooklyn            -1.5      105   
24    20171020    3     Sacramento         Dallas            -5.5      180   
25    20171020    3    L.A. Lakers        Phoenix            -3.0      130   
26    20171021    4   Philadelphia        Toronto            -7.5      415   
27    20171021    4         Dallas        Houston           -10.5      700   
28    20171021    4        Indiana          Miami            -8.5      425   
29    20171021    4   Golden State        Memphis             7.5     -440   
...        ...  ...            ...            ...             ...      ...   
1200  20180408  173        Detroit        Memphis             5.5     -220   
1201  20180408  173           Utah    L.A. Lakers             6.0     -325   
1202  20180408  173   Golden State        Phoenix            13.5    -1150   
1203  20180409  174        Toronto        Detroit             3.5     -175   
1204  20180409  174        Chicago       Brooklyn            -8.5      290   
1205  20180409  174        Memphis      Minnesota           -15.0     1000   
1206  20180409  174     Sacramento    San Antonio           -13.0      850   
1207  20180409  174      Cleveland       New York            12.0     -800   
1208  20180409  174  Oklahoma City          Miami             2.0     -140   
1209  20180409  174        Orlando      Milwaukee           -10.0      550   
1210  20180409  174       Portland         Denver            -3.5      155   
1211  20180409  174    New Orleans  L.A. Clippers             5.5     -255   
1212  20180410  175      Charlotte        Indiana            -6.5      240   
1213  20180410  175         Boston     Washington            -6.5      245   
1214  20180410  175   Golden State           Utah            -7.0      245   
1215  20180410  175   Philadelphia        Atlanta             9.5     -500   
1216  20180410  175        Phoenix         Dallas            -4.0      150   
1217  20180410  175        Houston    L.A. Lakers             9.5     -660   
1218  20180411  176        Toronto          Miami            -7.0      170   
1219  20180411  176      Milwauk

In [11]:
df["ml_bet_result"] = "N/A"

df.loc[(df.ml_bet == "home") & (df.result < 0), "ml_bet_result"] = "W"
df.loc[(df.ml_bet == "home") & (df.result > 0), "ml_bet_result"] = "L"
df.loc[(df.ml_bet == "away") & (df.result < 0), "ml_bet_result"] = "L"
df.loc[(df.ml_bet == "away") & (df.result > 0), "ml_bet_result"] = "W"

df

date  day           away           home  opening_spread  away_ML  \
0     20171017    0         Boston      Cleveland            -3.0      155   
1     20171017    0        Houston   Golden State            -9.0      385   
2     20171018    1       Brooklyn        Indiana            -3.5      145   
3     20171018    1          Miami        Orlando             3.0     -150   
4     20171018    1      Milwaukee         Boston            -5.5      205   
5     20171018    1        Atlanta         Dallas            -6.5      220   
6     20171018    1      Minnesota    San Antonio            -1.0      105   
7     20171018    1   Philadelphia     Washington            -7.0      255   
8     20171018    1      Charlotte        Detroit            -3.0      145   
9     20171018    1    New Orleans        Memphis            -3.0      135   
10    20171018    1         Denver           Utah            -2.0      110   
11    20171018    1        Houston     Sacramento             8.5     -360   
12    20171018    1       Portland        Phoenix             1.0     -123   
13    20171019    2        Chicago        Toronto           -11.0      750   
14    20171019    2       New York  Oklahoma City           -11.0      575   
15    20171019    2  L.A. Clippers    L.A. Lakers             4.5     -195   
16    20171020    3         Boston   Philadelphia             1.0      105   
17    20171020    3        Detroit     Washington            -6.5      200   
18    20171020    3       Portland        Indiana             3.5     -165   
19    20171020    3           Utah      Minnesota            -5.5      165   
20    20171020    3   Golden State    New Orleans             6.5     -320   
21    20171020    3        Atlanta      Charlotte            -6.0      200   
22    20171020    3      Cleveland      Milwaukee             1.5     -135   
23    20171020    3        Orlando       Brooklyn            -1.5      105   
24    20171020    3     Sacramento         Dallas            -5.5      180   
25    20171020    3    L.A. Lakers        Phoenix            -3.0      130   
26    20171021    4   Philadelphia        Toronto            -7.5      415   
27    20171021    4         Dallas        Houston           -10.5      700   
28    20171021    4        Indiana          Miami            -8.5      425   
29    20171021    4   Golden State        Memphis             7.5     -440   
...        ...  ...            ...            ...             ...      ...   
1200  20180408  173        Detroit        Memphis             5.5     -220   
1201  20180408  173           Utah    L.A. Lakers             6.0     -325   
1202  20180408  173   Golden State        Phoenix            13.5    -1150   
1203  20180409  174        Toronto        Detroit             3.5     -175   
1204  20180409  174        Chicago       Brooklyn            -8.5      290   
1205  20180409  174        Memphis      Minnesota           -15.0     1000   
1206  20180409  174     Sacramento    San Antonio           -13.0      850   
1207  20180409  174      Cleveland       New York            12.0     -800   
1208  20180409  174  Oklahoma City          Miami             2.0     -140   
1209  20180409  174        Orlando      Milwaukee           -10.0      550   
1210  20180409  174       Portland         Denver            -3.5      155   
1211  20180409  174    New Orleans  L.A. Clippers             5.5     -255   
1212  20180410  175      Charlotte        Indiana            -6.5      240   
1213  20180410  175         Boston     Washington            -6.5      245   
1214  20180410  175   Golden State           Utah            -7.0      245   
1215  20180410  175   Philadelphia        Atlanta             9.5     -500   
1216  20180410  175        Phoenix         Dallas            -4.0      150   
1217  20180410  175        Houston    L.A. Lakers             9.5     -660   
1218  20180411  176        Toronto          Miami            -7.0      170   
1219  20180411  176      Milwauk

In [12]:
w = len(df[df.spread_bet_result == "W"])
l = len(df[df.spread_bet_result == "L"])
p = len(df[df.spread_bet_result == "P"])

print("ATS RESULTS:")
print(str(w) + "-" + str(l) + "-" + str(p))
print(w/(w+l))

ATS RESULTS:
320-270-14
0.5423728813559322


In [13]:
w = len(df[df.ml_bet_result == "W"])
l = len(df[df.ml_bet_result == "L"])
p = len(df[df.ml_bet_result == "P"])

print("SU RESULTS:")
print(str(w) + "-" + str(l) + "-" + str(p))
print(w/(w+l))

SU RESULTS:
759-337-0
0.6925182481751825
